In [21]:
import numpy as np
import torch
import torch.nn as nn
from typing import Tuple
from torchvision.models.segmentation import deeplabv3_resnet50
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageFile
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [2]:
def compute_accuracy(predicted_batch: torch.Tensor, label_batch: torch.Tensor) -> float:
    pred = predicted_batch.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    acum = pred.eq(label_batch.view_as(pred)).sum().item()
    return acum

In [3]:
seed = 123
np.random.seed(seed)
_ = torch.manual_seed(seed)
_ = torch.cuda.manual_seed(seed)

In [4]:
# we select to work on GPU if it is available in the machine, otherwise will run on CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [5]:
# Define some hyper-parameters
hparams = {
    'batch_size': 20,
    'num_epochs': 10,
    'test_batch_size': 20,
    'learning_rate': 0.01,
    'weight_decay': 1e-5,
    'log_interval': 10,
}

In [6]:
import os

In [7]:
# Get the current working directory
current_dir = os.getcwd()

# Print the current working directory
current_dir

'/content'

In [8]:
# Get the path to the directory
initial_path = "/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/Wildfire Prediction Dataset (Satellite Images)"
dataset_path = "/dataset/wildfire_prediction"

# List the contents of the directory
contents = os.listdir(initial_path + dataset_path)

# Print the contents of the directory
for item in contents:
  print(item)

valid
train
test


In [9]:
initial_path

'/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/Wildfire Prediction Dataset (Satellite Images)'

In [10]:
dataset_path

'/dataset/wildfire_prediction'

In [11]:
contents

['valid', 'train', 'test']

In [12]:
# defining paths of train, validation and test data
train_path = initial_path + dataset_path + "/train"
test_path = initial_path + dataset_path + "/test"
valid_path = initial_path + dataset_path + "/valid"

print("Training data: " + train_path)
print("Test data: " + test_path)
print("Validation data: " + valid_path)

Training data: /content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/Wildfire Prediction Dataset (Satellite Images)/dataset/wildfire_prediction/train
Test data: /content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/Wildfire Prediction Dataset (Satellite Images)/dataset/wildfire_prediction/test
Validation data: /content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/Wildfire Prediction Dataset (Satellite Images)/dataset/wildfire_prediction/valid


In [13]:
# defining image transformations
image_transforms = transforms.Compose([
        transforms.Resize((350, 350)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5],
                             [0.5, 0.5, 0.5])
    ])

In [14]:
# loading training data using DataLoader
train_data = ImageFolder(train_path, transform=image_transforms)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=hparams['batch_size'], shuffle=True)

In [15]:
# loading test data using DataLoader
test_data = ImageFolder(test_path, transform=image_transforms)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=hparams['batch_size'], shuffle=False)

# loading validation data using DataLoader
val_data = ImageFolder(valid_path, transform=image_transforms)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=hparams['batch_size'], shuffle=False)

In [16]:
# Retrieve a sample from the dataset by simply indexing it
img, label = train_data[0]
print('Img shape: ', img.shape)
print('Label: ', label)

Img shape:  torch.Size([3, 350, 350])
Label:  0


In [17]:
# Sample a BATCH from the dataloader by running over its iterator
iter_ = iter(train_loader)
bimg, blabel = next(iter_)
print('Batch Img shape: ', bimg.shape)
print('Batch Label shape: ', blabel.shape)
print('Batch Img shape: ', bimg.shape)
print('Batch Label shape: ', blabel.shape)
print(f'The Batched tensors return a collection of {bimg.shape[0]} images \
({bimg.shape[1]} channel, {bimg.shape[2]} height pixels, {bimg.shape[3]} width \
pixels)')
print(f'In the case of the labels, we obtain {blabel.shape[0]} batched integers, one per image')

Batch Img shape:  torch.Size([256, 3, 350, 350])
Batch Label shape:  torch.Size([256])
Batch Img shape:  torch.Size([256, 3, 350, 350])
Batch Label shape:  torch.Size([256])
The Batched tensors return a collection of 256 images (3 channel, 350 height pixels, 350 width pixels)
In the case of the labels, we obtain 256 batched integers, one per image


In [18]:
# Get the class idx + names
class_names = train_data.class_to_idx
class_names

{'nowildfire': 0, 'wildfire': 1}

In [19]:
print("Train:")
print(f"Found {len(train_data)} images belonging to {train_data.classes} classes.")
print("Test:")
print(f"Found {len(test_data)} images belonging to {test_data.classes} classes.")
print("Val:")
print(f"Found {len(val_data)} images belonging to {val_data.classes} classes.")

Train:
Found 30250 images belonging to ['nowildfire', 'wildfire'] classes.
Test:
Found 6300 images belonging to ['nowildfire', 'wildfire'] classes.
Val:
Found 6300 images belonging to ['nowildfire', 'wildfire'] classes.


In [20]:
# Get the current directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

Current directory: /content


In [22]:
def _plot_learning_curves(train_losses, train_accs, val_losses, val_accs):
    plt.figure(figsize=(10, 8))
    plt.subplot(2, 1, 1)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.plot(train_losses, label='train')
    plt.plot(val_losses, label='eval')
    plt.legend()
    plt.subplot(2, 1, 2)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy [%]')
    plt.plot(train_accs, label='train')
    plt.plot(val_accs, label='eval')
    plt.show()  # Not needed when using 'agg'

In [23]:
def train_single_epoch(train_loader: torch.utils.data.DataLoader,
                       model: torch.nn.Module,
                       optimizer: torch.optim,
                       criterion: torch.nn.functional,
                       epoch: int,
                       log_interval: int,
                       ) -> Tuple[float, float]:
    # Activate the train=True flag inside the model
    model.train()

    train_loss = []
    acc = 0.
    avg_weight = 0.1
    for batch_idx, (data, target) in enumerate(train_loader):

        # Move input data and labels to the device
        data, target = data.to(device), target.to(device)

        # Set model gradients to 0.
        optimizer.zero_grad()

        # Forward batch of images through the model
        output = model(data)

        # Compute loss
        loss = criterion(output, target)

        # Compute backpropagation
        loss.backward()

        # Update parameters of the model
        optimizer.step()

        # Compute metrics
        acc += compute_accuracy(output, target)
        train_loss.append(loss.item())

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
    avg_acc = 100. * acc / len(train_loader.dataset)

    return np.mean(train_loss), avg_acc

In [24]:
@torch.no_grad()  # decorator: avoid computing gradients
def val_single_epoch(
        val_loader: torch.utils.data.DataLoader,
        model: torch.nn.Module,
        criterion: torch.nn.functional
        ) -> Tuple[float, float]:

    # Dectivate the train=True flag inside the model
    model.eval()

    eval_loss = []
    acc = 0
    for data, target in val_loader:
        data, target = data.to(device), target.to(device)

        output = model(data)

        # Apply the loss criterion and accumulate the loss
        eval_loss.append(criterion(output, target).item())

        # compute number of correct predictions in the batch
        acc += compute_accuracy(output, target)

    # Average accuracy across all correct predictions batches now
    eval_acc = 100. * acc / len(val_loader.dataset)
    eval_loss = np.mean(eval_loss)
    print('\Validation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        eval_loss, acc, len(val_loader.dataset), eval_acc,
        ))
    return eval_loss, eval_acc

In [25]:
def train_model(train_loader: torch.utils.data.DataLoader,
                val_loader: torch.utils.data.DataLoader,
                test_loader: torch.utils.data.DataLoader,
                hparams: dict
                ) -> deeplabv3_resnet50:
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []

    # Define pretrained ResNet-50 model using TorchVision DeepLabV3 architecture
    model = deeplabv3_resnet50(pretrained=True)

    # Since the segmented images have 2 classes (wildfire and nowildfire), we change the final layer to 2 classes
    model.classifier[4] = torch.nn.Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1))
    model = model.to(device)

    # Select Stochastic gradient descent (SGD) optimizer with a learning rate of 0.01 (optimum for image segmentation)
    optimizer = torch.optim.SGD(model.parameters(), lr=hparams['learning_rate'])
    criterion = nn.CrossEntropyLoss()

    for epoch in range(hparams['num_epochs']):
        # Compute & save the average training loss for the current epoch
        train_loss, train_acc = train_single_epoch(
            train_loader=train_loader,
            model=model,
            optimizer=optimizer,
            criterion=criterion,
            epoch=epoch,
            log_interval=hparams["log_interval"]
        )
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        val_loss, val_acc = val_single_epoch(
            val_loader=val_loader,
            model=model,
            criterion=criterion
        )
        val_losses.append(val_loss)
        val_accs.append(val_acc)

    final_test_loss, final_test_acc = test_model(
        test_loader=test_loader,
        model=model,
        criterion=criterion
    )

    # Plot the plots of the learning curves
    _plot_learning_curves(train_losses, train_accs, val_losses, val_accs)

    # Print or plot final test results
    print('Final Test set: Average loss: {:.4f}, Accuracy: {:.2f}%'.format(
        final_test_loss, final_test_acc))

    return model

In [26]:
model = train_model(train_loader=train_loader,
                    val_loader=val_loader,
                    test_loader=test_loader,
                    hparams=hparams).to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:01<00:00, 126MB/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.89 GiB. GPU 0 has a total capacty of 14.75 GiB of which 1.61 GiB is free. Process 3499 has 13.13 GiB memory in use. Of the allocated memory 11.34 GiB is allocated by PyTorch, and 1.67 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

In [ ]:
# Save the model checkpoint
save_path = "./model.pt"
print(f"Saving model to {save_path}...")
save_model(model, save_path)
print("Model saved successfully!")